# Feature Extraction

In [1]:
import pandas as pd
import numpy as np
import sys
import nltk
import warnings
warnings.simplefilter(action='ignore')

In [67]:
df = pd.read_csv('top_24_winery.csv',index_col = 0)
df = df[~df.price.isnull()].reset_index()
df.head()

def returnprice(price):
    if price<=20:
        return 'low'
    elif price<=50:
        return 'medium'
    else:
        return 'high'

df['price']=df['price'].apply(returnprice).map({'low':1,'medium':2,'high':3}).astype('category')
df.head()

,index,winery,description,points,price
0,0,Domaine Zind-Humbrecht,"Rich gold in color. Broad, layered aromas of v...",90,3
1,1,Testarossa,"Cooked cranberry is spiced with anise, pepperc...",91,3
2,2,Robert Mondavi,"Pithy, with grapefruit and lemon peel flavors,...",90,1
3,3,Testarossa,"Overly sweet and simple, and something of a di...",85,2
4,4,Robert Mondavi,"With rich, sweet blackberry and cocoa flavors,...",87,2


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4160 entries, 0 to 4159
Data columns (total 5 columns):
index          4160 non-null int64
winery         4160 non-null object
description    4160 non-null object
points         4160 non-null int64
price          4160 non-null category
dtypes: category(1), int64(2), object(2)
memory usage: 134.2+ KB


In [52]:
%run ./Text_Normalization_Function.ipynb

'C:\Users\Erin' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'C:\Users\Erin' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'C:\Users\Erin' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [53]:
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

In [54]:
norm_description = normalize_corpus(df.description)

In [55]:
print('RAW TEXT: ', np.array(df.description)[0])
print()
print('NORMALIZED TEXT: ', norm_description[0])

RAW TEXT:  Rich gold in color. Broad, layered aromas of very ripe fruit with hints of sweet smoke, brown sugar, honey and a savory earthy minerality. Full-bodied, bone dry, but richly textured with crisp acidity and a wide palette of flavors, ripe stone fruit, a creamy savory earthiness, sage and a balsamic kick on the finish. Very long length, slightly warm but with a lingering savory, mineral finish.

NORMALIZED TEXT:  rich gold color broad layer aroma ripe fruit hint sweet smoke brown sugar honey savory earthy minerality full bodied bone dry richly textured crisp acidity wide palette flavor ripe stone fruit creamy savory earthiness sage balsamic kick finish long length slightly warm linger savory mineral finish


In [56]:
df['description']=norm_description

In [57]:
target_names=df['winery'].unique()
target_names

array(['Domaine Zind-Humbrecht', 'Testarossa', 'Robert Mondavi',
       'Louis Latour', 'Undurraga', 'Chehalem', 'Iron Horse', 'Foxen',
       'Chateau Ste. Michelle', 'Wines & Winemakers', 'Georges Duboeuf',
       'Maryhill', 'Fess Parker', 'Santa Ema', 'Kendall-Jackson',
       'Williams Selyem', 'Concha y Toro', 'Trapiche',
       'Feudi di San Gregorio', 'Louis Jadot', 'DFJ Vinhos', 'V. Sattui',
       'Columbia Crest', 'Montes', 'Casa Santos Lima',
       'Chanson Père et Fils', 'Bründlmayer', 'Lynmar', 'Siduri', 'Kunde',
       'Gary Farrell', 'Jean-Luc and Paul Aegerter', 'Albert Bichot'],
      dtype=object)

**Please seperate the data into test dataset and training dataset up to this point.**

In [58]:
from scipy import sparse

In [70]:
def train_test_with_words(df):
    def get_train_test(df):
        test=df.sample(n=887, replace=False,axis=0)
        test_bunch['data']=test['description']
        test_bunch['target']=test['winery']
        test_bunch['price']=test['price']
        train=df.drop(axis=0, index=test.index)
        train_bunch['data']=train['description']
        train_bunch['target']=train['winery']
        train_bunch['price']=train['price']
    
        return test_bunch,train_bunch
    
    test_bunch={'data':[],'target':[],'price':[]}
    train_bunch={'data':[],'target':[],'price':[]}
    
    test,train=get_train_test(df)
    
    train_price_array = np.array(train['price'])
    train_price_csr_array = sparse.csr_matrix(train_price_array).transpose()
    test_price_array = np.array(test['price'])
    test_price_csr_array = sparse.csr_matrix(test_price_array).transpose()
        
    
    ## BOW train!!
    vectorizer_bow = CountVectorizer(stop_words = 'english') 
    feature_matrix_bow = vectorizer_bow.fit_transform(train['data']).astype(float) # replace norm_description by your training data
    feature_matrix_bow = sparse.hstack((feature_matrix_bow,train_price_csr_array))
    # 2-gram train!!!
    vectorizer_2grams = TfidfVectorizer(ngram_range = (2,2)) 
    feature_matrix_2grams = vectorizer_2grams.fit_transform(train['data']).astype(float) # replace norm_description by your training data
    feature_matrix_2grams = sparse.hstack((feature_matrix_2grams,train_price_csr_array))
    # 3-gram train!!!
    vectorizer_3grams = TfidfVectorizer(ngram_range = (3,3)) 
    feature_matrix_3grams = vectorizer_3grams.fit_transform(train['data']).astype(float) # replace norm_description by your training data
    feature_matrix_3grams = sparse.hstack((feature_matrix_3grams,train_price_csr_array))
    # for this part, please fit the testdata
    feature_matrix_TEST_bow = vectorizer_bow.transform(test['data'])
    feature_matrix_TEST_bow = sparse.hstack((feature_matrix_TEST_bow,test_price_csr_array))
    feature_matrix_TEST_2grams = vectorizer_2grams.transform(test['data'])
    feature_matrix_TEST_2grams = sparse.hstack((feature_matrix_TEST_2grams,test_price_csr_array))
    feature_matrix_TEST_3grams = vectorizer_3grams.transform(test['data'])
    feature_matrix_TEST_3grams = sparse.hstack((feature_matrix_TEST_3grams,test_price_csr_array))
    
    return test,train,feature_matrix_bow,feature_matrix_2grams,feature_matrix_3grams,feature_matrix_TEST_bow,feature_matrix_TEST_2grams,feature_matrix_TEST_3grams

## Model building

In [79]:
accuracy_2grams=[]
accuracy_3grams=[]
accuracy_bow=[]
accuracy_train_bow=[]
accuracy_train_3grams=[]
accuracy_train_2grams=[]

from scipy import sparse
for i in range (0,10):
    test,train,feature_matrix_bow,feature_matrix_2grams,feature_matrix_3grams,feature_matrix_TEST_bow,feature_matrix_TEST_2grams,feature_matrix_TEST_3grams=train_test_with_words(df)
    from sklearn.naive_bayes import MultinomialNB
    
    #Naive Bayes

    #using 2 grams
    clf_2grams = MultinomialNB(alpha=0.1) 
    clf_2grams.fit(feature_matrix_2grams, train['target'])
    predicted_nb_2grams = clf_2grams.predict(feature_matrix_TEST_2grams)
    cm_2grams = metrics.confusion_matrix(test['target'], predicted_nb_2grams)
    #Confusion_matrix=pd.DataFrame(data = cm, columns = target_names,index = target_names)
    accuracy_2grams.append(metrics.accuracy_score(test['target'], predicted_nb_2grams))
    accuracy_train_2grams.append(metrics.accuracy_score(train['target'], clf_2grams.predict(feature_matrix_2grams)))

    
    #using 3 grams
    clf_3grams = MultinomialNB(alpha=0.1) 
    clf_3grams.fit(feature_matrix_3grams, train['target'])
    predicted_nb_3grams = clf_3grams.predict(feature_matrix_TEST_3grams)
    cm_3grams = metrics.confusion_matrix(test['target'], predicted_nb_3grams)
    accuracy_3grams.append(metrics.accuracy_score(test['target'], predicted_nb_3grams))
    accuracy_train_3grams.append(metrics.accuracy_score(train['target'], clf_3grams.predict(feature_matrix_3grams)))

    
    #using bag of words
    clf_bow = MultinomialNB(alpha=0.1) 
    clf_bow.fit(feature_matrix_bow, train['target'])
    predicted_nb_bow = clf_bow.predict(feature_matrix_TEST_bow)
    cm_bow = metrics.confusion_matrix(test['target'], predicted_nb_bow)
    accuracy_bow.append(metrics.accuracy_score(test['target'], predicted_nb_bow))
    accuracy_train_bow.append(metrics.accuracy_score(train['target'], clf_bow.predict(feature_matrix_bow)))

    

print("Accuracy rate of using 2 grams: \n", np.mean(list(accuracy_2grams))) 
print("Accuracy rate of using 3 grams: \n", np.mean(list(accuracy_3grams))) 
print("Accuracy rate of using bag of words: \n", np.mean(list(accuracy_bow))) 
print("Accuracy rate of using bag of words for training data: \n", np.mean(list(accuracy_train_bow))) 
print("Accuracy rate of using 3 grams for training data: \n", np.mean(list(accuracy_train_3grams))) 
print("Accuracy rate of using 2 grams for training data: \n", np.mean(list(accuracy_train_2grams))) 

Accuracy rate of using 2 grams: 
 0.34543404735062005
Accuracy rate of using 3 grams: 
 0.2961668545659526
Accuracy rate of using bag of words: 
 0.48184892897406983
Accuracy rate of using bag of words for training data: 
 0.9667277726856096
Accuracy rate of using 3 grams for training data: 
 0.9989000916590284
Accuracy rate of using 2 grams for training data: 
 0.9871982890314698


In [80]:
accuracy_2grams=[]
accuracy_3grams=[]
accuracy_bow=[]
accuracy_train_bow=[]
accuracy_train_3grams=[]
accuracy_train_2grams=[]
for i in range (0,10):
    test,train,feature_matrix_bow,feature_matrix_2grams,feature_matrix_3grams,feature_matrix_TEST_bow,feature_matrix_TEST_2grams,feature_matrix_TEST_3grams=train_test_with_words(df)
    from sklearn import linear_model
    
    #Logit Regression

    #using 2 grams
    clf_2grams = linear_model.SGDClassifier(loss='log') 
    clf_2grams.fit(feature_matrix_2grams, train['target'])
    predicted_nb_2grams = clf_2grams.predict(feature_matrix_TEST_2grams)
    cm_2grams = metrics.confusion_matrix(test['target'], predicted_nb_2grams)
    #Confusion_matrix=pd.DataFrame(data = cm, columns = target_names,index = target_names)
    accuracy_2grams.append(metrics.accuracy_score(test['target'], predicted_nb_2grams))
    accuracy_train_2grams.append(metrics.accuracy_score(train['target'], clf_2grams.predict(feature_matrix_2grams)))

    
    #using 3 grams
    clf_3grams = linear_model.SGDClassifier(loss='log') 
    clf_3grams.fit(feature_matrix_3grams, train['target'])
    predicted_nb_3grams = clf_3grams.predict(feature_matrix_TEST_3grams)
    cm_3grams = metrics.confusion_matrix(test['target'], predicted_nb_3grams)
    accuracy_3grams.append(metrics.accuracy_score(test['target'], predicted_nb_3grams))
    accuracy_train_3grams.append(metrics.accuracy_score(train['target'], clf_3grams.predict(feature_matrix_3grams)))

    
    #using bag of words 
    clf_bow = linear_model.SGDClassifier(loss='log') 
    clf_bow.fit(feature_matrix_bow, train['target'])
    predicted_nb_bow = clf_bow.predict(feature_matrix_TEST_bow)
    cm_bow = metrics.confusion_matrix(test['target'], predicted_nb_bow)
    accuracy_bow.append(metrics.accuracy_score(test['target'], predicted_nb_bow))
    accuracy_train_bow.append(metrics.accuracy_score(train['target'], clf_bow.predict(feature_matrix_bow)))


print("Accuracy rate of using 2 grams: \n", np.mean(list(accuracy_2grams))) 
print("Accuracy rate of using 3 grams: \n", np.mean(list(accuracy_3grams))) 
print("Accuracy rate of using bag of words: \n", np.mean(list(accuracy_bow))) 
print("Accuracy rate of using bag of words for training data: \n", np.mean(list(accuracy_train_bow))) 
print("Accuracy rate of using 3 grams for training data: \n", np.mean(list(accuracy_train_3grams))) 
print("Accuracy rate of using 2 grams for training data: \n", np.mean(list(accuracy_train_2grams))) 

Accuracy rate of using 2 grams: 
 0.25050732807215337
Accuracy rate of using 3 grams: 
 0.21792559188275087
Accuracy rate of using bag of words: 
 0.43731679819616687
Accuracy rate of using bag of words for training data: 
 0.9867705468988694
Accuracy rate of using 3 grams for training data: 
 0.7898869538649557
Accuracy rate of using 2 grams for training data: 
 0.7516040329972502


In [81]:
accuracy_2grams=[]
accuracy_3grams=[]
accuracy_bow=[]
accuracy_train_bow=[]
accuracy_train_3grams=[]
accuracy_train_2grams=[]
for i in range (0,10):
    test,train,feature_matrix_bow,feature_matrix_2grams,feature_matrix_3grams,feature_matrix_TEST_bow,feature_matrix_TEST_2grams,feature_matrix_TEST_3grams=train_test_with_words(df)
    from sklearn import linear_model
    
    #SVM 

    #using 2 grams
    clf_2grams = linear_model.SGDClassifier(loss='hinge') 
    clf_2grams.fit(feature_matrix_2grams, train['target'])
    predicted_nb_2grams = clf_2grams.predict(feature_matrix_TEST_2grams)
    cm_2grams = metrics.confusion_matrix(test['target'], predicted_nb_2grams)
    #Confusion_matrix=pd.DataFrame(data = cm, columns = target_names,index = target_names)
    accuracy_2grams.append(metrics.accuracy_score(test['target'], predicted_nb_2grams))
    accuracy_train_2grams.append(metrics.accuracy_score(train['target'], clf_2grams.predict(feature_matrix_2grams)))

    
    #using 3 grams
    clf_3grams = linear_model.SGDClassifier(loss='hinge') 
    clf_3grams.fit(feature_matrix_3grams, train['target'])
    predicted_nb_3grams = clf_3grams.predict(feature_matrix_TEST_3grams)
    cm_3grams = metrics.confusion_matrix(test['target'], predicted_nb_3grams)
    accuracy_3grams.append(metrics.accuracy_score(test['target'], predicted_nb_3grams))
    accuracy_train_3grams.append(metrics.accuracy_score(train['target'], clf_3grams.predict(feature_matrix_3grams)))

    
    #using bag of words 
    clf_bow = linear_model.SGDClassifier(loss='hinge') 
    clf_bow.fit(feature_matrix_bow, train['target'])
    predicted_nb_bow = clf_bow.predict(feature_matrix_TEST_bow)
    cm_bow = metrics.confusion_matrix(test['target'], predicted_nb_bow)
    accuracy_bow.append(metrics.accuracy_score(test['target'], predicted_nb_bow))
    accuracy_train_bow.append(metrics.accuracy_score(train['target'], clf_bow.predict(feature_matrix_bow)))


print("Accuracy rate of using 2 grams: \n", np.mean(list(accuracy_2grams))) 
print("Accuracy rate of using 3 grams: \n", np.mean(list(accuracy_3grams))) 
print("Accuracy rate of using bag of words: \n", np.mean(list(accuracy_bow))) 
print("Accuracy rate of using bag of words for training data: \n", np.mean(list(accuracy_train_bow))) 
print("Accuracy rate of using 3 grams for training data: \n", np.mean(list(accuracy_train_3grams))) 
print("Accuracy rate of using 2 grams for training data: \n", np.mean(list(accuracy_train_2grams))) 

Accuracy rate of using 2 grams: 
 0.24193912063134163
Accuracy rate of using 3 grams: 
 0.2142051860202931
Accuracy rate of using bag of words: 
 0.4205186020293123
Accuracy rate of using bag of words for training data: 
 0.9620531622364803
Accuracy rate of using 3 grams for training data: 
 0.8009471432936145
Accuracy rate of using 2 grams for training data: 
 0.7598838985640086
